### Tecnologías de Gestión de Información No Estructurada
Master Universitario en Tecnologías de Análisis de Datos Masivos: Big Data
#### PRÁCTICA 1: Extrayendo y analizando web data sobre adicción al juego
La web (ya sea una red social o una página web) representa una importante fuente de
acceso a contenidos. En esta práctica, nos centraremos en scrapear contenido de
páginas web o redes sociales relacionado con gambling problems (adicción al juego, en
español). Este trabajo está orientado a familiarizarse con técnicas básicas de creación de
corpus textuales, así como en una primera exploración del contenido de los textos
mediante la extracción de términos centrales o importantes.

### Paso 1: Extracción de datos de la web

    - Nota: Para la recuperación web podéis usar librerías como Beautiful Soup o Scrapy y para recuperar de Reddit existe Praw

### 1.1 Conectarse a la URL de destino para hacer raspado web

In [1]:
import requests
from bs4 import BeautifulSoup
url_example='https://quotes.toscrape.com'
# peticiones para descargar una página web con la siguiente línea de código: 
# request.get() realiza una petición GET utilizando la URL pasada como parámetro. 
page = requests.get(url_example)
# Si la petición HTTP se ejecuta correctamente, page.status_code contendrá 200
# Un código de estado HTTP 4xx o 5xx representará un error. Esto puede ocurrir por varias razones, 
# pero tenga en cuenta que la mayoría de los sitios web bloquean las solicitudes que no contienen un encabezado User-Agent válido
print(page.status_code)
#Puede establecer una cabecera User-Agent válida en las peticiones de la siguiente manera:  

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

page = requests.get('https://quotes.toscrape.com', headers=headers)
print(page.status_code)

200
200


### 1.2 Extrayendo datos con el raspador web de Python
 
Para extraer datos de una página web, primero identifica los elementos HTML con la información deseada del DOM y busca los selectores CSS adecuados en las herramientas de desarrollo de tu navegador

    1.  Pasar el documento HTML 'page.text' al constructor BeautifulSoup() 
    2.  Extraer elementos del DOM
    3.  Iterar sobre la lista de citas para recuperar los datos de la cita
    4.  Transformar estos datos en un diccionario y añadirlos a la lista de citas 

In [2]:
# 1.  Pasar el documento HTML 'page.text' al constructor BeautifulSoup():  
# NOTA : El segundo parámetro especifica el analizador que Beautiful Soup utilizará para analizar el documento HTM
soup = BeautifulSoup(page.text, 'html.parser')

#  variable que contendrá la lista de todos los datos raspados.
quotes = []

#2.  extraer elementos del DOM  
# NOTA:  find_all() devolverá la lista de todos los elementos HTML <div> 
# identificados por la clase quote. Equivale a aplicar el selector CSS .quot
quote_elements = soup.find_all('div', class_='quote')

#3. iterar sobre la lista de citas para recuperar los datos de la cita 
# NOTA: Gracias al método find() de Beautiful Soup puede extraer el único elemento HTML de interés
for quote_element in quote_elements:
    # extracting the text of the quote
    text = quote_element.find('span', class_='text').text
    # extracting the author of the quote
    author = quote_element.find('small', class_='author').text

    # extracting the tag <a> HTML elements related to the quote
    tag_elements = quote_element.find('div', class_='tags').find_all('a', class_='tag')

    # storing the list of tag strings in a list
    tags = []
    for tag_element in tag_elements:
        tags.append(tag_element.text)
    #4.  Transformar estos datos en un diccionario y añadirlos a la lista de citas 

    quotes.append(
        {
            'text': text,
            'author': author,
            'tags': ', '.join(tags) # merging the tags into a "A, B, ..., Z" string
        }
    )
print(quotes)

[{'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', 'author': 'Albert Einstein', 'tags': 'change, deep-thoughts, thinking, world'}, {'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', 'author': 'J.K. Rowling', 'tags': 'abilities, choices'}, {'text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”', 'author': 'Albert Einstein', 'tags': 'inspirational, life, live, miracle, miracles'}, {'text': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”', 'author': 'Jane Austen', 'tags': 'aliteracy, books, classic, humor'}, {'text': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”", 'author': 'Marilyn Monroe', 'tags': 'be-yourself, inspirational'}, {'text': '“Try not to become

### 1.3 Implementación de la lógica de recopilación de datos

iterar sobre cada página hasta que no hay página siguiente. En concreto, extrae la URL relativa de la página siguiente y la utiliza para crear la URL de la siguiente página a rastrear. A continuación, descargue la página siguiente. Después, raspe y repita la lógica.  

In [3]:
# la URL de la pagina de inicio del sitio web objetivo
base_url = 'https://quotes.toscrape.com'

# recuperando la pagina e inicializando el objeto 'soup'...

# obteniendo el elemento HTML "Siguiente →"
next_li_element = soup.find('li', class_='next')

# si hay una siguiente pagina para extraer
while next_li_element is not None:
    next_page_relative_url = next_li_element.find('a', href=True)['href']

    # obteniendo la nueva pagina
    page = requests.get(base_url + next_page_relative_url, headers=headers)

    # analizando la nueva pagina
    soup = BeautifulSoup(page.text, 'html.parser')

    # logica de extraccion...

    # buscando el elemento HTML "Siguiente →" en la nueva pagina
    next_li_element = soup.find('li', class_='next')


### 1 Implementación para el ejercicio


In [4]:
import requests
from bs4 import BeautifulSoup

base_url='https://www.ludopatia.org/forum/default.asp'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

page = requests.get(base_url, headers=headers)
print(page.status_code)

200


In [5]:
# Definir la función para extraer información de un foro individual
def extraer_info_foro(url_foro):
    response = requests.get(url_foro)
    if response.status_code == 200:
        foro_soup = BeautifulSoup(response.text, 'html.parser')
        
        # Aquí debes escribir el código para extraer la información específica del foro,
        # siguiendo un proceso similar al que utilizaste para la página principal de foros.
        
        # Por ejemplo, si deseas extraer el título del foro:
        titulo_foro_elem = foro_soup.find('title')
        titulo_foro = titulo_foro_elem.text if titulo_foro_elem else "No disponible"
        
        # Otra información relevante del foro se puede extraer de manera similar.
        # Encontrar la tabla que contiene los posts
        tabla_posts = soup.find('table', attrs={'width': '100%', 'border': '0', 'cellspacing': '1', 'cellpadding': '3'})
        
        # Inicializar una lista para almacenar los datos de los posts
        datos_posts = []
        
        # Recorrer las filas de la tabla (ignorando la primera fila que contiene encabezados)
        filas = tabla_posts.find_all('tr')[1:]
        
        for fila in filas:
            # Extraer los datos de cada celda de la fila
            celdas = fila.find_all('td')
        
            # Comprobar que hay suficientes celdas para extraer la información
            if len(celdas) == 6:
                # Extraer la información relevante
                titulo = celdas[1].find('a').text
                autor = celdas[2].find('a').text
                respuestas = celdas[3].text
                vistas = celdas[4].text
                ultimo_mensaje = celdas[5].find('a').text
                hora = celdas[5].find('br').next_sibling.strip()  # Obtener la hora
        
                # Agregar los datos a la lista
                datos_posts.append({
                    'Titulo': titulo,
                    'Autor': autor,
                    'Respuestas': respuestas,
                    'Vistas': vistas,
                    'Último mensaje': ultimo_mensaje,
                    'Hora': hora
                })

        return titulo_foro,datos_posts  # Devuelve la información extraída
    else:
        print(f"No se pudo acceder a la URL del foro: {url_foro}")
        return None

# Definir la función para extraer información de un foro individual
def extraer_info_foro(url_foro):
    response = requests.get(url_foro)
    if response.status_code == 200:
        foro_soup = BeautifulSoup(response.text, 'html.parser')
        
        # Aquí debes escribir el código para extraer la información específica del foro,
        # siguiendo un proceso similar al que utilizaste para la página principal de foros.
        
        # Por ejemplo, si deseas extraer el título del foro:
        titulo_foro_elem = foro_soup.find('title')
        titulo_foro = titulo_foro_elem.text if titulo_foro_elem else "No disponible"
        
        # Otra información relevante del foro se puede extraer de manera similar.
        # Encontrar la tabla que contiene los posts
        tabla_posts = soup.find('table', attrs={'width': '100%', 'border': '0', 'cellspacing': '1', 'cellpadding': '3'})
        
        # Inicializar una lista para almacenar los datos de los posts
        datos_posts = []
        
        # Recorrer las filas de la tabla (ignorando la primera fila que contiene encabezados)
        filas = tabla_posts.find_all('tr')[1:]
        
        for fila in filas:
            # Extraer los datos de cada celda de la fila
            celdas = fila.find_all('td')
        
            # Comprobar que hay suficientes celdas para extraer la información
            if len(celdas) == 6:
                # Extraer la información relevante
                titulo = celdas[1].find('a').text
                autor = celdas[2].find('a').text
                respuestas = celdas[3].text
                vistas = celdas[4].text
                ultimo_mensaje = celdas[5].find('a').text
                hora = celdas[5].find('br').next_sibling.strip()  # Obtener la hora
        
                # Agregar los datos a la lista
                datos_posts.append({
                    'Titulo': titulo,
                    'Autor': autor,
                    'Respuestas': respuestas,
                    'Vistas': vistas,
                    'Último mensaje': ultimo_mensaje,
                    'Hora': hora
                })

        return titulo_foro,datos_posts  # Devuelve la información extraída
    else:
        print(f"No se pudo acceder a la URL del foro: {url_foro}")
        return None

In [6]:
### 1.1 Conectarse a la URL de destino para hacer raspado web
# URL base del sitio web
base_url = "https://www.ludopatia.org/forum/"

# Crear un objeto BeautifulSoup con la página principal de foros
url_principal_foros = base_url + "default.asp"  # Puedes ajustar la URL según tus necesidades
response = requests.get(url_principal_foros, headers=headers)


if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar todas las filas de la tabla que contiene la información de los foros
    filas_foros = soup.select('table[width="98%"] table[width="100%"] tr')
    
    # Iterar a través de las filas para extraer información de cada foro
    for fila in filas_foros[1:]:  # Omitimos la primera fila que contiene encabezados
        celdas = fila.find_all('td')
    
        if len(celdas) == 5:
            # Extraer información sobre el foro
            nombre_foro_elem = celdas[1].find('a')
            nombre_foro = nombre_foro_elem.text if nombre_foro_elem else "No disponible"
            temas = celdas[2].text
            mensajes = celdas[3].text
            ultimo_mensaje = celdas[4].text.strip()
            
            # Imprimir la información del foro
            print(f"Nombre del foro: {nombre_foro}")
            print(f"Número de temas: {temas}")
            print(f"Número de mensajes: {mensajes}")
            print(f"Último mensaje: {ultimo_mensaje}")
            print("-" * 50)
        
            # Extraer la URL del foro
            url_foro_elem = celdas[1].find('a')
            if url_foro_elem:
                url_foro = base_url + url_foro_elem['href']
                
                # Llamar a la función para extraer información del foro individual
                info_foro = extraer_info_foro(url_foro)
                if info_foro:
                    print(f"Información del foro ({url_foro}): {info_foro}")
                print("-" * 50)
else:
    print(f"No se pudo acceder a la página principal de foros: {url_principal_foros}")

Nombre del foro: No disponible
Número de temas: Temas
Número de mensajes: Mensajes
Último mensaje: Último mensaje
--------------------------------------------------
Nombre del foro: ASAMBLEA  FORA DE JOC
Número de temas: 1
Número de mensajes: 1
Último mensaje: 23 de Agosto de 2023 a las 23:16
        Por administrator
--------------------------------------------------
Información del foro (https://www.ludopatia.org/forum/forum_topics.asp?FID=16): ('Foros de Ludopatia.org: ASAMBLEA  FORA DE JOC', [])
--------------------------------------------------
Nombre del foro: ASAMBLEA  AAFAL
Número de temas: 1
Número de mensajes: 1
Último mensaje: 23 de Agosto de 2023 a las 23:20
        Por administrator
--------------------------------------------------
Información del foro (https://www.ludopatia.org/forum/forum_topics.asp?FID=17): ('Foros de Ludopatia.org: ASAMBLEA  AAFAL', [])
--------------------------------------------------
Nombre del foro: General de ayuda
Número de temas: 7974
Número de

In [54]:
### 1.1 Conectarse a la URL de destino para hacer raspado web
# URL base del sitio web
base_url = "http://foroapuestas.forobet.com/ludopatia-adiccion-y-problemas-con-el-juego/"

# Your HTML content goes here
html =  requests.get(url_principal_foros, headers=headers)

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Extract the title tag
title_tag = soup.find('title')
if title_tag:
    title = title_tag.text
    print("Title:", title)

# Extract meta tags with name="keywords" and name="description"
meta_keywords = soup.find('meta', attrs={'name': 'keywords'})
meta_description = soup.find('meta', attrs={'name': 'description'})

if meta_keywords:
    keywords = meta_keywords.get('content')
    print("Keywords:", keywords)
                       
if meta_description:
    description = meta_description.get('content')
    print("Description:", description)

# Extract all the links in the HTML
links = soup.find_all('a')
for link in links:
    href = link.get('href')
    text = link.text
    print("Link:", href, "| Text:", text)
# Crear un objeto BeautifulSoup con la páginf']
                
                # Llamar a la función para extraer información del foro individual
                info_foro = extraer_info_foro(url_foro)
                if info_foro:
                    print(f"Información del foro ({url_foro}): {info_foro}")



IndentationError: unexpected indent (2452569170.py, line 38)

In [55]:
soup = BeautifulSoup(page.text, 'html.parser')
#  print(soup)
#  variable que contendrá la lista de todos los datos raspados.
quotes = []


quote_elements = soup.find_all('div', class_='quote')
print(quote_elements)

# Encontrar todas las filas de la tabla que contiene la información de los foros
filas_foros = soup.select('table[width="98%"] table[width="100%"] tr')


# Iterar a través de las filas para extraer información de cada foro
for fila in filas_foros[1:]:  # Omitimos la primera fila que contiene encabezados
    celdas = fila.find_all('td')
    
    if len(celdas) == 5:
        # Extraer información sobre el foro
        nombre_foro_elem = celdas[1].find('a')
        nombre_foro = nombre_foro_elem.text if nombre_foro_elem else "No disponible"
        temas = celdas[2].text
        mensajes = celdas[3].text
        ultimo_mensaje = celdas[4].text.strip()
        
        # Imprimir la información del foro
        print(f"Nombre del foro: {nombre_foro}")
        print(f"Número de temas: {temas}")
        print(f"Número de mensajes: {mensajes}")
        print(f"Último mensaje: {ultimo_mensaje}")
        print("-" * 50)
       
        # Extraer la URL del foro
        url_foro_elem = celdas[1].find('a')
        if url_foro_elem:
            url_foro = '/'.join(base_url.split('/')[:-1])+ '/' +url_foro_elem['href']
            print(f"base_url del foro: {base_url},{url_foro_elem}")
            # Llamar a la función para extraer información del foro individual
            info_foro = extraer_info_foro(url_foro)
            if info_foro:
                        print(f"Información del foro ({url_foro}): {info_foro}")
            print("-" * 50)


[]
Nombre del foro: No disponible
Número de temas: Temas
Número de mensajes: Mensajes
Último mensaje: Último mensaje
--------------------------------------------------
Nombre del foro: ASAMBLEA  FORA DE JOC
Número de temas: 1
Número de mensajes: 1
Último mensaje: 23 de Agosto de 2023 a las 23:16
        Por administrator
--------------------------------------------------
base_url del foro: http://foroapuestas.forobet.com/ludopatia-adiccion-y-problemas-con-el-juego/,<a href="forum_topics.asp?FID=16" target="_self">ASAMBLEA  FORA DE JOC</a>
Información del foro (http://foroapuestas.forobet.com/ludopatia-adiccion-y-problemas-con-el-juego/forum_topics.asp?FID=16): ('Foro de apuestas deportivas 🏅| Forobet.com', [])
--------------------------------------------------
Nombre del foro: ASAMBLEA  AAFAL
Número de temas: 1
Número de mensajes: 1
Último mensaje: 23 de Agosto de 2023 a las 23:20
        Por administrator
--------------------------------------------------
base_url del foro: http://for

In [ ]:
http://foroapuestas.forobet.com/ludopatia-adiccion-y-problemas-con-el-juego/